In [2]:
import pandas as pd
import os

In [6]:
os.listdir("./ledgered/ledgered_app/test_resources/mint_transactions.csv")

['.DS_Store', 'mint_transactions.csv']

In [25]:


entries_df = pd.read_csv("./ledgered/ledgered_app/test_resources/mint_transactions.csv")
entries_df.columns = [
            "date",
            "description",
            "original_description",
            "amount",
            "transaction_type",
            "category",
            "account_name",
            "labels",
            "notes",
        ]
entries_df.rename(columns={"transaction_type":"entry_type"}, inplace=True)
entries_df["amount"] = entries_df["amount"].astype(float)

ROLLUP_KEYS = [
            "date",
            "entry_type",
            "original_description",
        ]

#print(entries_df)
#print(entries_df.info())
#print(entries_df.dtypes())
grouped_entries_df = entries_df.groupby(ROLLUP_KEYS, as_index=False).sum("amount")
#print(grouped_entries_df)

In [28]:
outfile = pd.read_csv("./ledgered/ledgered_app/test_resources/test_outfile.csv")
grouped_entries_df = outfile.groupby(ROLLUP_KEYS, as_index=False).sum("amount")

In [29]:
grouped_entries_df

,date,entry_type,original_description,amount,labels,notes
0,11/05/2021,debit,AUTH : TST* REVIVAL CAFE - DAVIS,16.86,0.0,0.0
1,11/09/2021,debit,AUTH : TST* THE BURREN,15.75,0.0,0.0
2,11/09/2021,debit,BFRESH 2420 SOMERVILLE MA,23.99,0.0,0.0
3,11/09/2021,debit,CASH ADVANCE ATM2407 102 PRINCE ST BOSTON MA11...,83.00,0.0,0.0
4,11/09/2021,debit,DAVE S FRESH PASTA SOMERVILLE MA,33.05,0.0,0.0
5,11/09/2021,debit,TITAN GAS STATION MEDFORD MA,30.00,0.0,0.0
6,11/10/2021,debit,AUTH : PNT*NINE BAR ESPRESSO,3.75,0.0,0.0
7,11/10/2021,debit,AUTH : REI.COM 800-426-4840,22.16,0.0,0.0
8,11/11/2021,debit,AUTH : DAVE S FRESH PASTA,12.44,0.0,0.0
9,11/11/2021,debit,AUTH : PNT*NINE BAR ESPRESSO,10.75,0.0,0.0
